In [38]:
import urllib.request
from urllib.request import urlopen as uReq
import re
from bs4 import BeautifulSoup as soup
import pandas as pd
import os
import requests
import multiprocessing
import time
import numpy as np
import sys

from multiprocessing import Pool
from joblib import Parallel, delayed

In [39]:
my_url = 'https://www.isdb.org/projects/data'
web='https://www.isdb.org'
hh='?page='
page_range=list(range(1))

manager = multiprocessing.Manager()
projectlist = manager.list()
ttable=manager.list()
big_table = pd.DataFrame()

In [40]:
def get_url (page):
    linklist=[]
    projectlist=[]
    for i in page:
        linklist.append(my_url+hh+str(i))
    return linklist

In [41]:
linklist=get_url(page_range)

In [42]:
def get_project (link):
    global projectlist
    r=requests.get(link)
    page_soup = soup(r.content, "html.parser")
    links = [a['href'] for a in page_soup.find_all('a', href=True)]
    for k in links:
        if ('/projects/data/' in k):
            projectlist.append(k)

    print('finished {}'.format(link))


In [43]:
Parallel(n_jobs=8)(delayed(get_project)(j) for j in linklist)

[None]

In [ ]:
project_final_list=list(map(lambda x: web+x,projectlist))

In [45]:
project_final_list[0]

'https://www.isdb.org/projects/data/uid-uif-322'

In [48]:
def get_everything (link):
    sys.setrecursionlimit(1000000)
    global ttable
    uClient = uReq(link)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    print('good')
    try:
        title = page_soup.find_all('h1')[-1].string
    except:
        title=np.nan
    try:
        brief = page_soup.find_all('div',{'class':['field--item']})[0].text.replace('\n',' ')
    except:
        brief=np.nan
    try:
        status = page_soup.find_all('div',{'class':['field field--name-name field--type-string field--label-hidden field--item']})[-1].text
    except:
        status=np.nan
    try:
        country = page_soup.find_all('div',{'class':['field field--name-name field--type-string field--label-hidden field--item']})[0].text
    except:
        country=np.nan
    try:
        sector = page_soup.find_all('div',{'class':['field field--name-name field--type-string field--label-hidden field--item']})[1].text
    except:
        sector=np.nan
    try:
        start = page_soup.find_all('div',{'class':['field field--name-isdb-project-start-date field--type-x-isdbproj field--label-above']})[0].text
        s_date = start.split('\n')[2]
    except:
        s_date=np.nan
    try:
        end = page_soup.find_all('div',{'class':['field field--name-isdb-project-end-date field--type-x-isdbproj field--label-above']})[0].text
        e_date = end.split('\n')[2]
    except:
        e_date=np.nan
    columns = ['Title', 'Brief', 'Status', 'Country', 'Sector', 'StartDate','EndDate']
    variable_list=[title, brief, status, country, sector, s_date, e_date]
    kkk=dict(zip(columns,variable_list))
    ttable.append(kkk)
    
    #table = pd.DataFrame([[title, brief, status, country, sector, s_date, e_date]],columns = ['Title', 'Brief', 'Status', 'Country', 'Sector', 'StartDate','EndDate'])
    #big_table=pd.concat([big_table,table],axis=0)
    print('finish')
    time.sleep(1)

In [49]:
start=time.time()
num_cores=multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(get_everything)(k) for k in project_final_list)
print('------time used: {} ------------'.format(time.time()-start))

------time used: 28.32391095161438 ------------


In [56]:
pd.DataFrame.from_dict(list(ttable))

,Brief,Country,EndDate,Sector,StartDate,Status,Title
0,Baraka Bank Syndication Sector: Finance (Trade...,Bahrain,27 March 2018,Finance,5 July 2578,Active,Baraka Bank Syndication
1,Al-Fareeda Residential Sector: Finance (Trade ...,Saudi Arabia,27 March 2018,Finance,5 July 2578,Active,Al-Fareeda Residential
2,ITFC Government of Djibouti Syndication Sector...,Djibouti,27 March 2018,Finance,16 July 2577,Active,ITFC Government of Djibouti Syndication
3,BMB TFC Portfolio Sector: Finance (Trade (Mura...,Turkey,27 March 2018,Finance,5 July 2578,Active,BMB TFC Portfolio
4,ITFC Government of Mauritania Syndication Sect...,Mauritania,27 March 2018,Finance,16 July 2577,Active,ITFC Government of Mauritania Syndication
5,ITFC Ecobank Syndication Sector: Finance (Trad...,Togo,27 March 2018,Finance,16 July 2577,Active,ITFC Ecobank Syndication
6,ITFC Government of Senegal Syndication Sector:...,Senegal,27 March 2018,Finance,16 July 2577,Active,ITFC Government of Senegal Syndication
7,Ewan Global Real Estate Co. (EGREC) Sector: Fi...,Saudi Arabia,27 March 2018,Finance,16 July 2577,Completed,Ewan Global Real Estate Co. (EGREC)
8,Sidra-Ancile Global Structured Trade Investmen...,Finance,27 March 2018,Active,16 July 2577,Active,Sidra-Ancile Global Structured Trade Investmen...
9,Ajlan Brothers and Co Sector: Finance (Others),Saudi Arabia,27 March 2018,Finance,16 July 2577,Active,Ajlan Brothers and Co


In [ ]:
#big_table.index=range(len(big_table))
big_table